# Computational Theory 

In [7]:
# for numerical data and methods 
import numpy as np 

## Problem 1: Binary Words and Operations


In [8]:
# Implement the following functions in Python. Use numpy to ensure that all variables and values are treated as 32-bit integers. These functions are defined in the Secure Hash Standard

## Problem 2: Fractional Parts of Cube Roots 


In [9]:
# Write a function called primes(n) that generates the first n prime numbers.

# Use the function to calculate the cube root of the first 64 primes.

# For each cube root, extract the first thirty-two bits of the fractional part.

# Display the result in hexadecimal.

# Test the results against what is in the Secure Hash Standard.

## Problem 3: Padding


In [10]:
# Write a generator function block_parse(msg) that processes messages according to section 5.1.1 and 5.2.1 of the Secure Hash Standard. 

## Problem 4: Hashes


In [11]:
# Write a function hash(current, block) that calculates the next hash value given the current hash value and the next message block according to section 6.2.2 SHA-256 Hash Computation on page 22 of the Secure Hash Standard.

## Problem 5: Passwords


In [12]:
# The following are the SHA-256 hashes of three common passwords that have been hashed using one pass of the SHA-256 algorithm. As strings, they were encoded using UTF-8. Determine the passwords and explain how you found them. 

## End